In [ ]:
from final_utils import *

In [ ]:
# defining loss for the model
MSE = tf.keras.losses.MeanSquaredError()

In [ ]:
def model_inceptionresnet_multigap(input_shape=None, 
                                   return_sizes=False, model_path='models/quality-mlsp-mtl-mse-loss.hdf5'): # this are the pretrained optimal weights for the feature extractor
    """
    Build InceptionResNetV2 multi-GAP model, that extracts narrow MLSP features.

    :param input_shape: shape of the input images
    :param return_sizes: return the sizes of each layer: (model, gap_sizes)
    :return: model or (model, gap_sizes)
    """
    model_base = InceptionResNetV2(weights='imagenet',
                                  include_top=False,
                                  input_shape=input_shape)

    model_base.load_weights(model_path)

    feature_layers = [l for l in model_base.layers if 'mixed' in l.name]
    gaps = [GlobalAveragePooling2D(name="gap%d" % i)(l.output)
           for i, l in enumerate(feature_layers)]
    concat_gaps = Concatenate(name='concatenated_gaps')(gaps)

    x = Dense(2048, kernel_initializer='he_normal', activation='relu')(concat_gaps)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(1024, kernel_initializer='he_normal', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(256, kernel_initializer='he_normal', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    pred = Dense(2, activation='softmax')(x)
    
    model = Model(inputs=model_base.input, outputs=pred)

    if return_sizes:
        gap_sizes = [np.int32(g.get_shape()[1]) for g in gaps]
        return (model, gap_sizes)
    else:
        return model


def data_loader(data, h=100, w=100):
    #big pictures require lots of computational data, so we resize them
    imgs = []
    skipped_image = []
    for path in data:
        try:
            img = Image.open(path).convert('RGB')
            img = img.resize((h, w), Image.Resampling.LANCZOS)
            img_tensor = tf.keras.utils.img_to_array(img)
            imgs.append(img_tensor)
        except:
            print('Skip')
    
#         if img.size > (1080,720):
#             x = img_to_array(img)
#             img.close()
#             img = resize_main(x, size = (None, 512))

    imgs = tf.stack(imgs, axis=0)
    imgs = tf.squeeze(imgs)
    
    if imgs.shape[2] == 3:
        imgs = imgs[None]

    return imgs


def trainer(model, data, weights_path, batch_size=32, epochs=30, learning_rate=0.03):
    X_train, X_test, y_train, y_test = data
   
    X_train_loaded = data_loader(X_train, h=100, w=100)
    X_test_loaded = data_loader(X_test, h=100, w=100)
    data_val = (X_test_loaded, y_test)
    print(f'{X_train_loaded.shape=}, {y_train.shape=}')
    
    model.load_weights(weights_path) 

    model.compile(loss=MSE,
                  optimizer=keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, 
                                              epsilon=1e-07, decay=0, amsgrad=False))

    checkpoint = keras.callbacks.ModelCheckpoint(weights_path, 
                                                 monitor='val_loss', 
                                                 verbose=1, 
                                                 save_best_only=True, 
                                                 mode='min')

    schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0)
    callbacks_list = [checkpoint, schedule]

    history = model.fit(X_train_loaded, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0,
                        callbacks=callbacks_list,
                        validation_data = data_val)

    return history

In [ ]:
#Choosing the number of image paths to read 
data_bad = np.array(glob('training-dataset/bad/*')[:1000])
data_good = np.array(glob('training-dataset/good/*')[:1000])

### Creating model, and loading weights

In [ ]:
model = model_inceptionresnet_multigap()
weights_path = f'models/mg_dense_26_07.hdf5' 
# model.save_weights(weights_path) #if we want to cancel learning and start from 0
model.load_weights(weights_path)

## Training

In [ ]:
batch_size = 8
epochs = 20

histories = []


data = get_train_pairs(data_bad, data_good, train_size=0.9, shuffle=True)

history = trainer(model, data, weights_path, batch_size=batch_size, epochs=epochs)
histories.append(history)
acc = calc_acc(model, weights_path, data[1], data[3])  #from final_utils.py


save_history(history, f'histories/{batch_size}_{epochs}')
print('-----Accuracy =', acc, ' -----')


## Version using tf.keras.preprocessing.image_dataset_from_directory 

In [ ]:
# batch_size= 8
# img_height = 100
# img_width = 100


# ds_train = tf.keras.preprocessing.image_dataset_from_directory(
#     folder_path,
#     labels="inferred",
#     label_mode="int",  # categorical, binary
#     # class_names=['0', '1', '2', '3', ...]
#     color_mode="rgb",
#     batch_size=batch_size,
#     image_size=(img_height, img_width),  # reshape if not in this size
#     shuffle=True,
#     seed=123,
#     validation_split=0.1,
#     subset="training",
# )

# ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
#     folder_path,
#     labels="inferred",
#     label_mode="int",  # categorical, binary
#     # class_names=['0', '1', '2', '3', ...]
#     color_mode="rgb",
#     batch_size=batch_size,
#     image_size=(img_height, img_width),  # reshape if not in this size
#     shuffle=True,
#     seed=123,
#     validation_split=0.1,
#     subset="validation",
# )

# model = model_inceptionresnet_multigap()

# model.compile(
#     optimizer=keras.optimizers.Adam(),
#     loss=[keras.losses.SparseCategoricalCrossentropy(from_logits=True),],
#     metrics=["accuracy"],
# )
# model.fit(ds_train, epochs=10, verbose=2)